In [260]:
#pip install geopy

In [262]:
#pip install geopandas

In [264]:
#pip install xlrd

In [257]:
import pandas as pd
import numpy as np
import json
from geopy.distance import geodesic 
import geopandas as gpd
import math

In [265]:
df = pd.read_csv("Blocks Data - DD_sample_data.csv")

In [266]:
df_zips = pd.read_excel("ZIP_Locale_Detail.xls")
#https://postalpro.usps.com/ZIP_Locale_Detail

In [267]:
#import the county-zip mapping file (US Census)
df_county = gpd.read_file("PVS_23_v2_ccd_12011.shp")

Streamline Zip Codes for location mapping

In [268]:
#make sure all zips are 5 digit strings
def zips(i):
    y = str(i)
    if len(y) == 3:
        x = ''.join(("00",y))
    elif len(y) == 4:
        x = ''.join(("0",y))
    else:
        x = y
    return x

In [269]:
df_zips["PHYSICAL ZIP_CHANGE"] = df_zips["PHYSICAL ZIP"].apply(lambda x: zips(x))

In [270]:
#convert extras from string to dictionary
df['extras'] = df['extras'].apply(lambda x: json.loads(x))

Create location fields for mapping and calculate distance between clean voting and collection location fields

In [271]:
def concat(x,y):
    if pd.isna(x):
        x = 0
    if pd.isna(y):
        y = 0
    return f"{x},{y}"

In [272]:
df['voting_location'] = df.apply(lambda row: concat(row['voting_address_latitude'], row['voting_address_longitude']), axis=1)

In [274]:
df['collection_location'] = df.apply(lambda row: concat(row['collection_location_latitude'], row['collection_location_longitude']), axis=1)

In [304]:
def dist(x, y):
    return round(geodesic(eval(x),eval(y)).miles,1)


In [303]:
df_clean_location_["distance"] = df_clean_location_.apply(lambda row: dist(row['voting_location'], row['collection_location']), axis=1)

In [276]:
#extract dictionary into df with separate columns for each key
df_extras = df.extras.apply(pd.Series)

In [277]:
#merge original df with dictionary df
df_clean = pd.merge(right=df, left=df_extras, right_index = True, left_index=True)

In [278]:
#merge with streamlined zip codes for voting location
df_clean_location = pd.merge(left=df_clean, right=df_zips[["PHYSICAL ZIP", "PHYSICAL CITY"]], left_on = "voting_zipcode", right_on = "PHYSICAL ZIP")

In [279]:
df_clean_location.rename(columns={"PHYSICAL CITY": "voting_city_clean"}, inplace=True)

In [280]:
df_clean_location = df_clean_location.drop(["PHYSICAL ZIP"], axis=1)

In [281]:
#merge with streamlined zip codes for collection location
df_clean_location_ = pd.merge(left=df_clean_location, right=df_zips[["PHYSICAL ZIP", "PHYSICAL CITY"]], left_on = "collection_location_zip", right_on = "PHYSICAL ZIP")

In [282]:
df_clean_location_.rename(columns={"PHYSICAL CITY": "collection_city_clean"}, inplace=True)

Clean up disparate versions for all dem fields extracted from Extras

In [286]:
#combine gender columns
gender_columns = [col for col in df_clean_location_.columns if 'gender' in col]

In [287]:
df_clean_location_['gender_combined'] = df_clean_location_[gender_columns].bfill(axis=1).iloc[:, 0]

In [288]:
#combine race columns
race_columns = [col for col in df_clean_location_.columns if 'race' in col]

In [289]:
df_clean_location_['race_combined']=df_clean_location_[race_columns].bfill(axis=1).iloc[:, 0]

In [290]:
#combine birth location columns
birth_combined = [col for col in df_clean_location_.columns if 'birth' in col]

In [291]:
df_clean_location_['birthplace_combined']=df_clean_location_[birth_combined].bfill(axis=1).iloc[:, 0]

In [292]:
df_clean_location_['gender']=df_clean_location_['gender_combined']
df_clean_location_['race and ethnicity'] = df_clean_location_['race_combined']
df_clean_location_['birthplace'] = df_clean_location_['birthplace_combined']
df_clean_location_.drop(columns=['gender_combined', 'race_combined','birthplace_combined'], inplace=True)

Clean up County Columns

In [293]:
#create a clean lookup file (Zip Codes to Counties)
df_zips_counties = pd.read_csv("Zips and Counties - Sheet1.csv", header=None, names=["data"])

In [294]:
df_zips_counties['zips_filter']=df_zips_counties['data'].str.isnumeric()
df_zips_counties['counties_filter']=df_zips_counties['data'].str.contains(" County")

zips = df_zips_counties[df_zips_counties['zips_filter']==True]
counties = df_zips_counties[df_zips_counties['counties_filter']==True]


In [295]:
# combine zips and counties 
zips.reset_index(drop=True, inplace=True)
counties.reset_index(drop=True, inplace=True)
df_zips_counties_combined = pd.concat([zips,counties], axis=1)

In [296]:
df_zips_counties_combined.columns.values[0]='zip_code'
df_zips_counties_combined.columns.values[3]='county'

In [297]:
df_zips_counties_combined['zip_code']=df_zips_counties_combined['zip_code'].astype(float)

In [298]:
#apply clean zip code file to full dataset
df_clean_location__ = pd.merge(left=df_clean_location_, right=df_zips_counties_combined, left_on = "voting_zipcode", right_on = "zip_code")

In [299]:
df_clean_location__.rename(columns={"county": "voting_county"}, inplace=True)

In [300]:
df_clean_location__['voting_street_address_one'] = df_clean_location__['voting_county']

df_clean_location__.drop(columns=[
  'zip_code', 'zips_filter',
       'counties_filter', 'voting_county', 'zips_filter', 'counties_filter'], inplace=True)

In [301]:
#create csv
df_clean_location__.to_csv("clean_blocks.csv", index=False)